# LTL-f BASED-TRACE ALIGNMENT

From constraint to LTL-f: http://www.diag.uniroma1.it/degiacom/papers/2014/AAAI14.pdf

From LTL-f to DFA: http://ltlf2dfa.diag.uniroma1.it/

From LTL-f to automaton: https://github.com/whitemech/logaut

LTL2DFA library: https://github.com/whitemech/LTLf2DFA/

In [1]:
import re
from typing import Match, cast

from ltlf2dfa.parser.ltlf import LTLfParser
from dataclasses import dataclass
from typing import Dict, Match, Set, Tuple, cast

## Constraint automaton

**CONSTRAINTS**

- [x] Chain precedence activity 16 - 17
- [x] Existence activity 1
- [x] Precedence activity 9 -10
- [x] Responded existence activity 5 - 6
- [x] Chain response activity 14 - 15
- [x] Not co-existence activity 19 -20
- [x] Not succession activity 20 -21
- [x] Not chain succession activity 22 - 23
- [x] Response activity 11 - 12
- [x] Absence2 activity 2


In [2]:
parser = LTLfParser()


constraints_10 = ["(!(q) & G((X(q) -> p)))", 
                    "F(a)", 
                    "((!(j) U i) | G(!(j)))", 
                    "(F(e) -> F(f))", 
                    "G((n -> X(o)))",
                    "!((F(s) & F(t)))",
                    "G((t -> !(F(u))))",
                    "G((v <-> !(X(w))))",
                    "G(k -> F(l))",
                    "!(F((b & X(F(b)))))"]

constraints_15 = ["(!(q) & G((X(q) -> p)))", 
                    "F(a)", 
                    "((!(j) U i) | G(!(j)))", 
                    "(F(e) -> F(f))", 
                    "G((n -> X(o)))",
                    "!((F(s) & F(t)))",
                    "G((t -> !(F(u))))",
                    "G((v <-> !(X(w))))",
                    "G(k -> F(l))",
                    "!(F((b & X(F(b)))))",
                    "(!(r) & G((X(r) -> q)))",
                    "G(h -> F(i))",
                    "F(j)",
                    "F(c)",
                    "G((o -> X(p)))"
                    ]  
constraints_20 = ["(!(q) & G((X(q) -> p)))", 
                    "F(a)", 
                    "((!(j) U i) | G(!(j)))", 
                    "(F(e) -> F(f))", 
                    "G((n -> X(o)))",
                    "!((F(s) & F(t)))",
                    "G((t -> !(F(u))))",
                    "G((v <-> !(X(w))))",
                    "G(k -> F(l))",
                    "!(F((b & X(F(b)))))",
                    "(!(r) & G((X(r) -> q)))",
                    "G(h -> F(i))",
                    "F(j)",
                    "F(c)",
                    "G((o -> X(p)))",
                    "((!(y) U x) | G(!(y)))",
                    "F(x)",
                    "G(c -> F(d))",
                    "!(F((g & X(F(g)))))",
                    "((!(m) U l) | G(!(m)))"]    

constraint_formulas = constraints_10 


In [3]:
# MONA OUTPUT:
class StructureOutput():
    def __init__(self, nb_states, variable_names, initial_state, accepting_states, rejecting_states, transitions): 
        self.nb_states = nb_states
        self.variable_names = variable_names
        self.initial_state = initial_state
        self.accepting_states = accepting_states
        self.rejecting_states = rejecting_states
        self.transitions = transitions


class StructureOutputRegex():
    """A wrapper to the textual output of MONA."""

    def __init__(self, output):
        self.output = output

    def total_function (self):
        def create_transitions(self) -> Dict[int, Dict[int, Set[str]]]:
            raw_transitions: Dict[int, Dict[int, Set[str]]] = {}
            lines = self.output.splitlines()
            # from the 8th line, the output specifies the transitions.
            transition_strings = lines[7:]
            for t in transition_strings:
                match = cast(
                    Match, re.search("State ([0-9]+): ([01X]+|) -> state ([0-9]+)", t)
                )
                if match is None:
                    continue
                start_state = int(match.group(1))
                guard = match.group(2)
                end_state = int(match.group(3))
                raw_transitions.setdefault(start_state, {}).setdefault(
                    end_state, set()
                ).add(guard)
            return raw_transitions

        variable_names : Tuple[str, ...] = tuple(cast(Match, re.search("DFA for formula with free variables: (.*)", self.output),).group(1).split())
        initial_state : int = int(cast(Match, re.search("Initial state: (.*)\n", self.output)).group(1))
        accepting_states : Set[int] = set(map(int,cast(Match, re.search("Accepting states: (.*)\n", self.output)).group(1).split(),))
        rejecting_states : Set[int] = set(map(int,cast(Match, re.search("Rejecting states: (.*)\n", self.output)).group(1).split(),))
        nb_states : int = int(cast(Match,re.search(r"Automaton has ([0-9]+) state(\(?s\)?)? and .* BDD-node(\(?s\)?)?",self.output,),).group(1))
        raw_transitions : Dict[int, Dict[int, Set[str]]] = create_transitions(self)
        

        return StructureOutput(nb_states, variable_names, initial_state, accepting_states, rejecting_states, raw_transitions)

In [4]:
index = 1
index_states = 0

all_automata = {}

for f in constraint_formulas:
    build_automaton = {}
    build_automaton["formula"] = f

    all_states_constr = []
    final_states_constr = []
    init_states_constr = []
    rejecting_states_constr = []
    rho_constr_basic = []
    rho_tobe_negated = []

    ## Parser + dfa #####################################################
    formula = parser(f)       # returns an LTLfFormula
    dfa = formula.to_dfa(mona_dfa_out=True)
    mona_output = StructureOutputRegex(output=dfa).total_function() 
   
    ## save all what is needed ##########################################
    init_states_constr = 's'+str(index_states)
    
    states = set.union(mona_output.rejecting_states, mona_output.accepting_states)
    states = list(states)
    for elem in states:
        all_states_constr.append('s'+str(elem+index_states))

    for elem in list(mona_output.accepting_states):
        final_states_constr.append('s'+str(elem+index_states))

    alphabet = mona_output.variable_names

    for key,elem in mona_output.transitions.items():
        s_start = key + index_states
        for k,e in elem.items():
            s_end = k + index_states
            sum = 0
            comb = list(e)[0]
            for char in comb:
                if(char!='X'):
                    sum += int(char)
            if(sum > 1):
                continue
            elif(sum == 1):
                index1 = comb.find('1')
                rho_constr_basic.append("s"+str(s_start)+" "+alphabet[index1]+" s"+str(s_end))
            elif(sum == 0):
                indeces0 = [i for i in range(len(comb)) if comb[i] in '0']
                res_list = [alphabet[i] for i in indeces0]
                rho_tobe_negated.append([s_start,s_end,res_list])                 


    build_automaton["all_states"] = all_states_constr
    build_automaton["final_states"] = final_states_constr
    build_automaton["init_state"] = init_states_constr
    build_automaton["transitions"] = rho_constr_basic
    build_automaton["symbols_constr"] = alphabet
    build_automaton["negated_transitions"] = rho_tobe_negated

    all_automata["a"+str(index)] = build_automaton

    index += 1
    index_states += len(all_states_constr)

## Load xes file
containing the trace

In [5]:
log_paths = {}
base_opt = "base"
syn_real = "synthetic" # or "real-life"
n_constr = ["10", "15", "20"]
constr_inverted = ["3", "4", "6"]
len_traces = ["1-50", "51-100", "101-150", "151-200"]

for i in range(len(n_constr)):
    for j in range(len(constr_inverted)):
        for k in range(len(len_traces)):
            key = n_constr[i]+"/"+constr_inverted[j]+"/"+len_traces[k]
            log_paths[key] = "dataset/logs/"+syn_real+"-logs/"+n_constr[i]+"constraints/"+constr_inverted[j]+" constraints inverted/log-from-"+n_constr[i]+"constr-model-"+constr_inverted[j]+"constr_inverted-"+len_traces[k]+".xes"

In [6]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
voc = {}
for i in range(len(alphabet)):
    voc[i+1] = alphabet[i]

def convertNumberToChar (val):
    return voc[val]

In [7]:
# Legge dal file .xes, estrapola le tracce, trasforma il valore delle tracce in caratteri,
# infine aggiungile al log sottoforma di stringhe
def readLog (log_path):
    # Inizializzazione variabili
    flag = False;           # Indica quando dobbiamo leggere un evento dal file
    trace = [];             # Lista di eventi sottoforma di interi
    traceChar = [];         # Lista di eventi sottoforma di char
    traceString = "";       # Stringa composta da eventi sottoforma di char
    log = []               # Lista di tracce ognuna delle quali è una stringa di char

    # Apriamo il file e leggiamolo riga per riga
    f = open  (log_path)
    f1 = f.readlines()
    
    # Per ogni riga del file...
    for x in f1:
        # Se c'è un evento, attiviamo la flag
        if (x.__contains__("<event>")):
            flag = True
        # Se flag attiva e siamo sulla riga dove è presente il nome dell'evento,
        # estrapoliamo il nome dell'evento e appendiamolo a trace
        if (flag and x.__contains__('<string key="concept:name"')):
            val = x.split('value="activity ',1)[1]
            val = val.split('"')[0]                
            trace.append(val)
            flag = False
        # Quando non ci sono più eventi possiamo lavorare sulla traccia in questione
        if (x.__contains__("</trace>")):
            for event in trace:
                traceChar.append(convertNumberToChar(int(event)))  # Converti gli eventi in char 
            traceString = "".join(traceChar)                       # Lista di eventi -> stringa
            log.append(traceString)                                # Appendi stringa a log

            # Inizializza nuovamente le variabili
            trace = []
            traceChar = []
            traceString = ""
    return log

## Trace automaton

In [8]:
all_traces_log = log_paths.copy()

for k, log_path in log_paths.items():
    traces_log = readLog(log_path) # list of traces

    all_traces = []

    for t in traces_log:
        trace = {}
        
        symbols_trace = list(set(t))
        
        ## Build trace automaton ###################################
        rho_trace_basic = []
        Q_trace = []
        for i in range(len(t)):
            Q_trace.append('t'+str(i))
            rho_trace_basic.append('t'+str(i)+" "+t[i] +" "+ 't'+str(i+1))

        Q_trace.append('t'+str(len(t)))
        init_state_trace = Q_trace[0]
        final_state_trace = Q_trace[-1]

        trace["symbols_trace"] = symbols_trace
        trace["init_state_trace"] = init_state_trace
        trace["final_state_trace"] = final_state_trace
        trace["Q_trace"] = Q_trace
        trace["rho_trace_basic"] = rho_trace_basic

        all_traces.append(trace)

        ## Now we add the transitions for the negated symbols ######

        for ID,automaton in all_automata.items():
            symb_constr = set(automaton["symbols_constr"])
            symb_trace = set(symbols_trace)
            all_symbs = list(set.union(symb_constr, symb_trace))

            trans = automaton["transitions"]
            for t_neg in automaton["negated_transitions"]:
                symbs = all_symbs.copy()
                for t in t_neg[2]:
                    symbs.remove(t)
                for elem in symbs:
                    if ("s"+str(t_neg[0])+" "+elem+" s"+str(t_neg[1]) not in trans):
                        trans.append("s"+str(t_neg[0])+" "+elem+" s"+str(t_neg[1]))

            automaton["transitions"] = trans
    all_traces_log[k] = all_traces

## PDDL

In [9]:
# step 1: unire tutte le celle seguenti in un'unica cella
# step 2: ciclare per ogni database e poi per ogni traccia
# step 3: ricordati di inserire il salvataggio nel ciclo

all_states_constr = []
for ID,a in all_automata.items():
    for elem in a['all_states']:
        all_states_constr.append(elem) 

for key,all_traces in all_traces_log.items():
    n_cons, inv, lenght = key.split('/')
    type_db = n_cons+"_"+inv+"_"+lenght

    for trace in all_traces:
        symbols_tot = []
        for ID,a in all_automata.items():
            for s in a["symbols_constr"]:
                symbols_tot.append(s)
        symbols_tot = list(set(symbols_tot).union(set(trace["symbols_trace"])))
        
        #DOMAIN ###########################################################################
        domain_name = "domain_"+base_opt+"_"+type_db

        pddl_domain_initial = "(define (domain "+domain_name+") "\
                        "(:requirements :strips :typing :action-costs) "\
                        "(:types trace_state automaton_state - state activity automaton_name) "

        pddl_domain_predicates = "(:predicates (trace ?t1 - trace_state "\
                                    "?e - activity "\
                                    "?t2 - trace_state) "\
                                    "(automaton "\
                                    "?s1 - automaton_state "\
                                    "?e - activity "\
                                    "?s2 - automaton_state) "\
                                    "(cur_state ?s - state) "\
                                    "(final_state ?s - state)) "\
                                    "(:functions (total-cost)) "


        pddl_domain_actions =  "(:action sync "\
                                ":parameters (?t1 - trace_state ?e - activity ?t2 - trace_state) "\
                                ":precondition (and (cur_state ?t1) (trace ?t1 ?e ?t2)) "\
                                ":effect (and (not (cur_state ?t1)) (cur_state ?t2) "\
                                "(forall (?s1 ?s2 - automaton_state) "\
                                "(when (and (cur_state ?s1) "\
                                "(automaton ?s1 ?e ?s2)) "\
                                "(and (not (cur_state ?s1)) "\
                                "(cur_state ?s2)))))) "\
                                "(:action add "\
                                ":parameters (?e - activity ?s1 ?s2 - state) "\
                                ":effect (and (increase (total-cost) 1) "\
                                "(forall (?s1 ?s2 - automaton_state) "\
                                "(when (and (cur_state ?s1) "\
                                "(automaton ?s1 ?e ?s2)) "\
                                "(and (not (cur_state ?s1)) "\
                                "(cur_state ?s2)))))) "\
                                "(:action del "\
                                ":parameters (?t1 - trace_state ?e - activity "\
                                "?t2 - trace_state) "\
                                ":precondition (and (cur_state ?t1) (trace ?t1 ?e ?t2)) "\
                                ":effect (and (increase (total-cost) 1) "\
                                "(not (cur_state ?t1)) (cur_state ?t2))))"

        pddl_domain = pddl_domain_initial + pddl_domain_predicates + pddl_domain_actions
        
        # PROBLEM ###########################################################################
        problem_name = "problem_"+base_opt+"_"+type_db
        pddl_problem_initial = "(define (problem "+problem_name+") (:domain "+domain_name+") "
        pddl_problem_objects = "(:objects "

        for q in trace["Q_trace"]:
            pddl_problem_objects += q+" "
        pddl_problem_objects += "- trace_state "


        for q in all_states_constr:
            pddl_problem_objects += q+" "
        pddl_problem_objects += "- automaton_state "


        for s in symbols_tot:
            pddl_problem_objects += s+" "
        pddl_problem_objects += "- activity"

        pddl_problem_objects += ") "


        pddl_problem_init = "(:init (= (total-cost) 0) (cur_state "+trace["init_state_trace"]+") "
        for t in trace["rho_trace_basic"]:
            pddl_problem_init += "(trace "+t+") "
        pddl_problem_init += "(final_state "+trace["final_state_trace"]+") "

        for ID, a in all_automata.items():
            pddl_problem_init += "(cur_state "+a["init_state"]+") " 
            for t in a["transitions"]:
                pddl_problem_init += "(automaton "+t+") "

            for i in range(len(a["final_states"])):
                pddl_problem_init += "(final_state "+a["final_states"][i]+") "
        pddl_problem_init += ") "


        pddl_problem_goal = "(:goal (forall (?s - state) "\
                            "(imply (cur_state ?s) (final_state ?s)))) "
        pddl_problem_metric = "(:metric minimize (total-cost)))"
        pddl_problem = pddl_problem_initial + pddl_problem_objects + pddl_problem_init + pddl_problem_goal + pddl_problem_metric

        # SAVE #####################################################################
        file1 = open("PDDL/"+syn_real+"/"+base_opt+"/"+domain_name+".pddl", "w")
        file1.write(pddl_domain)
        file1.close()

        file2 = open("PDDL/"+syn_real+"/"+base_opt+"/"+problem_name+".pddl", "w")
        file2.write(pddl_problem)
        file2.close()

Fast-downward planner:
`./fast-downward.py domain_multi.pddl problem_multi.pddl --search "astar(blind())"`